In [3]:
import pandas as pd
import numpy as np
import glob

from matplotlib import pyplot as plt
import seaborn as sns

import re
import string

from tqdm import tqdm

import transformers
# from transformers import BertModel, BertConfig, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import networkx as nx
from node2vec import Node2Vec

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report

import gensim
import dill

## Checking available devices

In [4]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0']


## Merging all csv files

In [5]:
filenames = glob.glob("russian-troll-tweets/*.csv")

li = []
for filename in filenames:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
dfs = pd.concat(li, axis=0, ignore_index=True)

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,15,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
dfs.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,...,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,...,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,...,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,...,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,...,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN


In [7]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2946207 entries, 0 to 2946206
Data columns (total 21 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   external_author_id  object
 1   author              object
 2   content             object
 3   region              object
 4   language            object
 5   publish_date        object
 6   harvested_date      object
 7   following           int64 
 8   followers           int64 
 9   updates             int64 
 10  post_type           object
 11  account_type        object
 12  retweet             int64 
 13  account_category    object
 14  new_june_2018       int64 
 15  alt_external_id     object
 16  tweet_id            int64 
 17  article_url         object
 18  tco1_step1          object
 19  tco2_step1          object
 20  tco3_step1          object
dtypes: int64(6), object(15)
memory usage: 472.0+ MB


## Keeping English tweets

In [8]:
dfs = dfs[dfs['language'] == 'English']
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2116867 entries, 0 to 2946181
Data columns (total 21 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   external_author_id  object
 1   author              object
 2   content             object
 3   region              object
 4   language            object
 5   publish_date        object
 6   harvested_date      object
 7   following           int64 
 8   followers           int64 
 9   updates             int64 
 10  post_type           object
 11  account_type        object
 12  retweet             int64 
 13  account_category    object
 14  new_june_2018       int64 
 15  alt_external_id     object
 16  tweet_id            int64 
 17  article_url         object
 18  tco1_step1          object
 19  tco2_step1          object
 20  tco3_step1          object
dtypes: int64(6), object(15)
memory usage: 355.3+ MB


In [9]:
dfs['account_type'].value_counts()

Right         704953
local         458012
Left          422141
Hashtager     236092
news          138581
Commercial    112580
Russian        20015
Koch           10759
?               6945
German          3567
Italian         1799
Arabic          1030
ZAPOROSHIA       172
Spanish           79
French            70
Ebola             70
Portuguese         2
Name: account_type, dtype: int64

In [10]:
dfs['account_category'].value_counts()

RightTroll      704953
NewsFeed        596593
LeftTroll       422141
HashtagGamer    236092
Commercial      112580
NonEnglish       26562
Fearmonger       11001
Unknown           6945
Name: account_category, dtype: int64

### Drop NaN

In [11]:
dfs.dropna(how='any', subset=['author', 'content'], inplace=True)
print(dfs.shape)

(2116866, 21)


# Creating 'hashtags' and 'mentions'  columns

### temp_df with 'author' and 'content'

In [12]:
temp_df = dfs[['author', 'content']]
temp_df.head()

,author,content
0,10_GOP,"""We have a sitting Democrat US Senator on tria..."
1,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...
2,10_GOP,Daughter of fallen Navy Sailor delivers powerf...
3,10_GOP,JUST IN: President Trump dedicates Presidents ...
4,10_GOP,"19,000 RESPECTING our National Anthem! #StandF..."


## 'hashtags' column

- #### Collecting hashtags from content

In [13]:
temp_df['hashtags'] = dfs['content'].str.findall(r'#.*?(?=\s|$)')
temp_df.head()

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,author,content,hashtags
0,10_GOP,"""We have a sitting Democrat US Senator on tria...",[]
1,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,[]
2,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,[#BoycottNFL]
3,10_GOP,JUST IN: President Trump dedicates Presidents ...,[]
4,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",[#StandForOurAnthem🇺🇸]


## 'mentions' column

- #### Collecting mentions from content

In [14]:
temp_df['mentions'] = dfs['content'].str.findall(r'@.*?(?=\s|$)')
temp_df.head()

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,author,content,hashtags,mentions
0,10_GOP,"""We have a sitting Democrat US Senator on tria...",[],[@nedryun]
1,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,[],[]
2,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,[#BoycottNFL],[]
3,10_GOP,JUST IN: President Trump dedicates Presidents ...,[],[]
4,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",[#StandForOurAnthem🇺🇸],[]


# Graph requirements

- U2H
 + dictionary contains {'author' : list of all his unique hashtags}
 + List of unique authors
 + List of unique hashtags
 
- U2M
 + dictionary contains {'author' : list of all his unique mentions}
 + List of unique authors
 + List of unique mentions

## U2H

#### - Creating dictionary contains {'author' : list of all his unique hashtags}

In [49]:
hashtags_dic = dict()
for i in tqdm(range(temp_df.shape[0]), position=0):
    hashtags_dic.setdefault(temp_df.iloc[i, 0], []).append(temp_df.iloc[i, 2])

100%|█████████████████████████████████████████████████████████████████████| 2116866/2116866 [00:26<00:00, 79616.15it/s]


In [50]:
print("author: ", list(hashtags_dic.keys())[0])
print("\nhashtags: ", list(hashtags_dic.values())[0])
print("\n# hashtags:", len(list(hashtags_dic.values())[0]))

author:  10_GOP

hashtags:  [[], [], ['#BoycottNFL'], [], ['#StandForOurAnthem🇺🇸'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['#HipHopAwards'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['#Iran'], [], [], ['#MAGA'], [], [], [], ['#WomenBoycottTwitter'], [], [], [], [], [], [], [], [], [], [], ['#FakeNews'], [], [], [], [], ['#LasVegas'], ['#LasVegas.'], [], ['#LasVegasShooting'], [], ['#LasVegasStrong🇺🇸'], ['#LasVegasShooting'], [], [], ['#VegasStrong'], [], ['#VegasStrong'], ['#BoycottNFL'], ['#SanJuan', '#TakeAKnee'], [], ['#FakeNews'], [], [], [], [], ['#PRStrong'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['#NFLBoycott', '#MAGA'], [], [], [], [], [], [], [], [], ['#IrmaHurricane2017'], ['#HurrcaneIrma'], [], ['#FightCensorship', '#FreeSpeech', '#LiberalAttack'], ['#HurricaneIrma', '#Irma', '#TampaReady', '#FLORIDAHURRICANE'], ['#Irma', '#IrmaHurricane2017'], [], ['#HurricaneIrma', '#HurrcaneIrma'], 

#### - Flatting the hashtags lists for each author

In [51]:
for i, (k, v) in enumerate(hashtags_dic.items()):
    hashtags_dic[k] = set([item for sublist in list(hashtags_dic.values())[i] for item in sublist])

In [52]:
print("author: ", list(hashtags_dic.keys())[0])
print("\nhashtags: ", list(hashtags_dic.values())[0])
print("\n# hashtags:", len(list(hashtags_dic.values())[0]))

author:  10_GOP

hashtags:  {'#IRMA', '#LasVegas', '#LasVegasStrong🇺🇸', '#Manafort', '#FEMA', '#EMMY2017', '#TropicalStormIrma', '#dotard', '#TrumpTrain', '#TakeAKnee', '#WeStand', '#PrayForMexico', '#MOAR.', '#MOAR', '#dogs', '#NRA', '#NeverForget', '#FightCensorship', '#HipHopAwards', '#HurrcaneIrma', '#Irma', '#FreeSpeech', '#LasVegasShooting', '#WhatHappened', '#DACADeal', '#Iran', '#NFLBoycott', '#60Minutes', '#NeverForget911', '#TuesdayThoughts', '#ColinKaepernick:', '#HurricaneIrma', '#CNNisFakeNews', '#HerpesHillary', '#BoycottNFL', '#LiberalAttack', '#TaxReform.', '#IrmaHurricane2017', '#BenAtBerkeley', "#MAGA'", '#Maddow', '#CrookedHillary', '#1A', '#AfterIrma', '#IfIWerePresidentForOneDay', '#MAGA', '#FLORIDAHURRICANE', '#TampaReady', '#TeamScalise', '#StandForOurAnthem🇺🇸', '#FreeTennessee', '#JJNonPoliticalTweet', '#WomenBoycottTwitter', '#LasVegas.', '#PRStrong', '#STLVerdict', '#SanJuan', "#AmericaFirst'", '#Berkeley', '#September11th', '#FakeNews', '#ConstitutionDay', '#

#### - Creating a list with all unique hashtags

In [53]:
hashtags_list = [item for sublist in list(temp_df['hashtags']) for item in sublist]

In [54]:
print(len(hashtags_list))
unique_hashtags_list = set(hashtags_list)
print(len(unique_hashtags_list))

1457604
110097


## U2M

#### - Creating dictionary contains {'author' : list of all his unique mentions}

In [15]:
mentions_dic = dict()
for i in tqdm(range(temp_df.shape[0]), position=0):
    mentions_dic.setdefault(temp_df.iloc[i, 0], []).append(temp_df.iloc[i, 3])

100%|█████████████████████████████████████████████████████████████████████| 2116866/2116866 [00:26<00:00, 80614.01it/s]


In [16]:
print("author: ", list(mentions_dic.keys())[0])
print("\nhashtags: ", list(mentions_dic.values())[0])
print("\n# hashtags:", len(list(mentions_dic.values())[0]))

author:  10_GOP

hashtags:  [['@nedryun'], [], [], [], [], [], [], ['@SenatorMenendez', '@CarmenYulinCruz'], [], ['@CNN'], ['@CNN'], ['@thehill'], [], ['@MichelleObama,'], ['@MichelleObama,'], [], ['@FLOTUS!'], [], ['@Breaking911'], [], [], [], [], [], [], [], [], [], [], [], [], ['@realDonaldTrump'], ['@realDonaldTrump', '@POTUS'], [], [], [], [], [], [], [], [], [], [], [], ['@Joy_Villa'], [], [], [], [], ['@Joy_Villa:'], [], [], [], ['@realDonaldTrump'], ['@realDonaldTrump'], ['@realDonaldTrump'], ['@realDonaldTrump'], ['@realDonaldTrump'], ['@realDonaldTrump'], ['@realDonaldTrump'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['@Cernovich', '@KFILE'], [], [], [], [], [], [], [], [], ['@CNN'], [], ['@AP_Politics'], ['@POTUS', '@FLOTUS'], [], [], [], [], [], ['@PrisonPlanet'], ['@PrisonPlanet', '@GrrrGraphics,one'], [], [], [], [], [], [], [], [], [], [], ['@YouTube', '@TeamYouTube,'], ['@SteveScalise'], [], [], [], [], [], [], [], [], ['@10_gop'], [], [], ['@10_gop'], []

#### - Flatting the mentions lists for each author

In [17]:
for i, (k, v) in enumerate(mentions_dic.items()):
    mentions_dic[k] = set([item for sublist in list(mentions_dic.values())[i] for item in sublist])

In [18]:
print("author: ", list(mentions_dic.keys())[0])
print("\nhashtags: ", list(mentions_dic.values())[0])
print("\n# hashtags:", len(list(mentions_dic.values())[0]))

author:  10_GOP

hashtags:  {'@HillaryClinton', '@NaeguNugu', '@netanyahu', '@5sahandful', '@AP_Politics', '@Joy_Villa', '@Cernovich', '@Thewall77515902', '@Joy_Villa:', '@PrisonPlanet', '@LauraLoomer', '@tamaraleighllc', '@RedPillBlack:', '@GIT_ER_DONE_USA', '@realDonaldTrump', '@POTUS', '@SteveScalise', '@SLMPD', '@DonnaShelley2', '@wahrbear', '@UN', '@ELEVEN_GOP', '@realTEN_GOP', '@FoxNews', '@nedryun', '@Roni_K_Patriot', '@jojoh888', '@Marines4MAGA', '@umknicken', '@10_gop.', '@benshapiro', '@SenatorMenendez', '@SteveScalise!', '@exposes_racism', '@CNN', '@ShopFloorNAM', '@ChaVonZee16', '@Breaking911', '@GrrrGraphics,one', '@brianstelter,', '@10_gop', '@YouTube', '@FLOTUS', '@CarmenYulinCruz', '@thehill', '@MichelleObama,', '@KFILE', '@JackPosobiec', '@larryelder', '@TEN_GOP', '@FLOTUS!', '@SebGorka', '@TeamYouTube,'}

# hashtags: 53


#### - Creating a list with all unique mentions

In [19]:
mentions_list = [item for sublist in list(temp_df['mentions']) for item in sublist]

In [20]:
print(len(mentions_list))
unique_mentions_list = set(mentions_list)
print(len(unique_mentions_list))

1043278
566628


# Contructing U2H and U2M graphs

In [21]:
import networkx as nx
# U2H = nx.Graph()
U2M = nx.Graph()

### Adding nodes

#### -  'author' nodes

In [22]:
# U2H.add_nodes_from(hashtags_dic.keys())
U2M.add_nodes_from(mentions_dic.keys())

#### - 'hashtags' and 'mentions' nodes

In [23]:
# U2H.add_nodes_from(unique_hashtags_list)
U2M.add_nodes_from(unique_mentions_list)

### Creating edges

#### - U2H edges

In [ ]:
for i, (k, v) in enumerate(hashtags_dic.items()):
    for h in v:
        U2H.add_edge(k, h)

#### - U2M edges

In [24]:
for i, (k, v) in enumerate(mentions_dic.items()):
    for h in v:
        U2M.add_edge(k, h)

# Fitting U2H and U2M graphs
We give node2vec.Node2Vec a networkx.Graph instance, and after using .fit() (which accepts any parameter accepted by we get a gensim.models.Word2Vec) we get in return a gensim.models.Word2Vec instance.

## U2H

### - Creating node2vec instance

In [ ]:
# Generate walks
node2vec = Node2Vec(U2H, dimensions=128, walk_length=10, num_walks=10)

### - Fitting the model

In [ ]:
# Learn embeddings 
model = node2vec.fit(window=10, min_count=1)

In [ ]:
model.save("U2H_model.model")

In [296]:
model = gensim.models.Word2Vec.load("U2H_model.model")

### - Inspect the similarity

In [297]:
for node, _ in model.most_similar('#Saudi'):
    print(node)

#Yemen
#US
#Libya
#Aleppo
#SaudiArabia
#UN
#Putin
#NASA
#Palestine
#UAE


C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


## Logistic regression

In [55]:
vec_dic = dict()
for i, (k, v) in enumerate(hashtags_dic.items()):
    vec_dic[k] = model.wv.get_vector(k)

In [56]:
list(vec_dic.values())[0]

array([-6.5027815e-01,  1.7860985e+00, -9.9835724e-02, -1.4523559e+00,
       -8.4916103e-01, -2.6196465e+00, -1.2881172e+00, -1.0082791e+00,
        1.4479263e+00,  2.5722644e+00, -2.6185408e+00, -1.1167067e+00,
       -8.0396116e-01, -1.9550753e-01, -9.2765123e-01, -1.4171256e+00,
       -2.4721653e+00,  2.9045898e-01, -1.7112731e+00, -1.0120827e+00,
        1.1989537e+00, -9.9787414e-02,  8.4593946e-01,  6.7469082e+00,
        5.9217703e-01,  2.2312825e+00,  4.0348172e+00,  2.0909085e+00,
       -1.0738090e+00,  1.1811446e+00, -6.1611813e-01, -2.7363998e-01,
       -6.3329142e-01, -1.0270088e+00, -2.2638702e-01,  1.5096796e-01,
        1.6666461e+00, -1.6182579e+00,  5.3560179e-01, -4.9096098e+00,
        2.3494067e+00, -6.2780923e-01, -1.9175055e+00,  1.7578743e-01,
        2.4549271e-01,  2.4559669e+00,  4.2887168e+00,  1.9157932e+00,
       -1.6679986e+00,  1.3591440e+00, -6.0485286e-01,  1.4566317e+00,
        2.9361257e-01,  2.1966298e+00,  2.6562362e+00,  1.3344505e-01,
      

In [57]:
df_unique = dfs[['author', 'account_category']].drop_duplicates()
df_unique['vec'] = list(vec_dic.values())
df_unique

,author,account_category,vec
0,10_GOP,RightTroll,"[-0.65027815, 1.7860985, -0.09983572, -1.45235..."
424,1D_NICOLE_,Fearmonger,"[-0.24397607, 0.56576157, 0.029521903, 0.28719..."
468,1ERIK_LEE,RightTroll,"[-0.14608952, 0.58423156, 0.1004195, 0.0661785..."
497,1LORENAFAVA1,NonEnglish,"[-3.2753332, 2.4318128, 1.6246526, 0.36988232,..."
3521,2NDHALFONION,RightTroll,"[-0.003391954, 0.0016707064, -0.00091963925, 0..."
...,...,...,...
2930731,PLSCALLMEABBY,Unknown,"[-0.16937305, 0.91246253, 0.51490873, 0.810634..."
2931021,PLUSHEVYEA,NonEnglish,"[-0.0032627394, -0.0017309841, 7.090016e-05, -..."
2931252,POLICESTATEME,LeftTroll,"[-1.5918598, 3.4651847, -1.4340701, 1.9369287,..."
2931482,POLIGRAPHME,Unknown,"[-2.4041529, 2.741313, -2.8414426, 2.6523516, ..."


In [58]:
df_unique = df_unique.loc[df_unique['account_category'].isin(['RightTroll', 'LeftTroll', 'NewsFeed'])]

In [59]:
df_unique

,author,account_category,vec
0,10_GOP,RightTroll,"[-0.65027815, 1.7860985, -0.09983572, -1.45235..."
468,1ERIK_LEE,RightTroll,"[-0.14608952, 0.58423156, 0.1004195, 0.0661785..."
3521,2NDHALFONION,RightTroll,"[-0.003391954, 0.0016707064, -0.00091963925, 0..."
5118,4EVER_SUSAN,RightTroll,"[-0.055862095, 0.3251371, 0.52855974, 1.076397..."
5181,4MYSQUAD,LeftTroll,"[-4.1108074, 1.354283, 1.74966, 0.9054484, 0.9..."
...,...,...,...
2906225,PHOENIXDAILYNEW,NewsFeed,"[-0.72074896, 3.8408182, -0.35335833, 2.361104..."
2920984,PIGEONTODAY,RightTroll,"[-0.0922146, 3.55546, 3.3696609, 0.0013206998,..."
2930145,PIPERTRTR,RightTroll,"[0.0033570186, -0.0021629774, -0.0023819532, 8..."
2930714,PITTSESTE,RightTroll,"[0.11633294, 0.355998, 0.13684948, -0.14894602..."


In [292]:
X = list(df_unique['vec'])
y = df_unique['account_category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [293]:
lg = LogisticRegression(penalty='l2')
lg.fit(X_train,y_train)
y_pred = lg.predict(X_test)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [294]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   LeftTroll       0.78      0.86      0.82        21
    NewsFeed       0.86      0.86      0.86         7
  RightTroll       0.97      0.94      0.95        64

    accuracy                           0.91        92
   macro avg       0.87      0.88      0.88        92
weighted avg       0.92      0.91      0.91        92



## U2M

### - Creating node2vec instance

In [63]:
# Generate walks
U2M_node2vec = Node2Vec(U2M, dimensions=128, walk_length=10, num_walks=10, temp_folder='temp_folder')


Computing transition probabilities:   0%|▏                                    | 2226/568789 [00:51<98:17:02,  1.60it/s]

KeyboardInterrupt: 

### - Fitting the model

In [ ]:
# Learn embeddings 
U2M_model = U2M_node2vec.fit(window=10, min_count=1)

### - Inspect the similarity

In [ ]:
for node, _ in U2M_model.most_similar('@TeamYouTube'):
    print(node)

In [ ]:
model.save("U2H_model.model")

# BERT

In [177]:
import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer

In [178]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
#     s = " ".join([word for word in s.split()
#                   if word not in stopwords.words('english')
#                   or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [179]:
bert_df = dfs[['author', 'content', 'account_category']].loc[dfs['account_category'].isin(['RightTroll', 'LeftTroll', 'NewsFeed'])]
bert_df

,author,content,account_category
0,10_GOP,"""We have a sitting Democrat US Senator on tria...",RightTroll
1,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,RightTroll
2,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,RightTroll
3,10_GOP,JUST IN: President Trump dedicates Presidents ...,RightTroll
4,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",RightTroll
...,...,...,...
2931477,POLICESTATEME,DUDE DROPS COP #bloodmoon #supermoon #depress...,LeftTroll
2931478,POLICESTATEME,The Police against the crowd #raw #love #LOL ...,LeftTroll
2931479,POLICESTATEME,The choice is yours #got7 #FifthHarmony #self...,LeftTroll
2931480,POLICESTATEME,F*ck the police! #edit #relatable #anime #edi...,LeftTroll


In [180]:
bert_df['content'].apply(text_preprocessing)

0          we have a sitting democrat us senator on trial...
1          marshawn lynch arrives to game in anti trump s...
2          daughter of fallen navy sailor delivers powerf...
3          just in president trump dedicates presidents c...
4          19 000 respecting our national anthem standfor...
                                 ...                        
2931477    dude drops cop bloodmoon supermoon depression ...
2931478    the police against the crowd raw love lol funn...
2931479    the choice is yours got7 fifthharmony selfiefo...
2931480    f ck the police edit relatable anime edit lol ...
2931481    cop shoots man reaching for drivers license co...
Name: content, Length: 1723687, dtype: object

In [181]:
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in tqdm(data, position=0):
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True,      # Return attention mask
            truncation=True
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [256]:
X = bert_df['content']
y = df_unique['account_category']

In [147]:
# Store the model we want to use
MODEL_NAME = "bert-large-uncased"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [35]:
encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in tqdm(bert_df['content'], position=0)]

100%|██████████████████████████████████████████████████████████████████████| 1723687/1723687 [10:52<00:00, 2640.27it/s]


In [159]:
print('Max length: ', max([len(sent) for sent in encoded_tweets]))

Max length:  347


In [197]:
MAX_LEN = 200

In [198]:
token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 91.15it/s]

Original:  "We have a sitting Democrat US Senator on trial for corruption and you've barely heard a peep from the mainstream media." ~ @nedryun https://t.co/gh6g0D1oiC
Token IDs:  [101, 2057, 2031, 1037, 3564, 7672, 2149, 5205, 2006, 3979, 2005, 7897, 1998, 2017, 2310, 4510, 2657, 1037, 21392, 2361, 2013, 1996, 7731, 2865, 16770, 1056, 2522, 1043, 2232, 2575, 2290, 2692, 2094, 2487, 19419, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [199]:
# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
X_inputs, X_masks = preprocessing_for_bert(X)

  0%|                                                                          | 165/1723687 [00:00<17:32, 1638.05it/s]

Tokenizing data...


100%|██████████████████████████████████████████████████████████████████████| 1723687/1723687 [14:04<00:00, 2041.88it/s]


In [237]:
len(X_inputs[0])

200

In [238]:
bert_dic = dict()
for i in tqdm(range(bert_df.shape[0]), position=0):
    bert_dic.setdefault(bert_df.iloc[i, 0], []).append(X_inputs[i].numpy())

100%|█████████████████████████████████████████████████████████████████████| 1723687/1723687 [00:21<00:00, 81454.06it/s]


#### Averaging the vectors

In [240]:
for k in bert_dic:
    bert_dic[k] = np.mean(bert_dic[k], keepdims=True)

#### Flatting the vector list

In [251]:
for i, (k, v) in enumerate(bert_dic.items()):
    bert_dic[k] = [item for sublist in list(bert_dic.values())[i] for item in sublist]

In [263]:
X = list(bert_dic.values())
y = df_unique['account_category']

In [289]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2020)

In [290]:
lg = LogisticRegression(penalty='l2')
lg.fit(X_train,y_train)
y_pred = lg.predict(X_test)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [291]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   LeftTroll       0.29      0.15      0.20        67
    NewsFeed       0.56      0.28      0.37        18
  RightTroll       0.71      0.86      0.78       191

    accuracy                           0.65       276
   macro avg       0.52      0.43      0.45       276
weighted avg       0.60      0.65      0.61       276

